<head>
  <meta name="author" content="Rogério de Oliveira">
  <meta institution="author" content="ITM">
</head>

<img src="https://maua.br/images/selo-60-anos-maua.svg" width=300, align="right">
<!-- <h1 align=left><font size = 6, style="color:rgb(200,0,0)"> optional title </font></h1> -->



# **P1 - RA PAR**

In [ ]:
#@markdown Nome completo e RA (*\<nome\>, \<RA\>*)
Aluno = 'Igor Improta Martinez da Silva, 21.00834-5' #@param {type:"string"}






In [3]:
path = 'https://github.com/Rogerio-mack/work/raw/main/data/'

import numpy as np
import pandas as pd

# **Case**: Airbnb

Neste case, analise dados sobre imóveis alugados pela plataforma Airbnb em NYC no ano de 2019.


In [4]:
df = pd.read_csv(path + 'AB_NYC_2019.csv')
df.head()

id                                            name  host_id  \
0   20913              Charming 1 bed GR8 WBurg LOCATION!    79402   
1   25235               Large 2 Bedroom Great for Groups!    87773   
2   76761  Eveland the Place to Stay & Enjoy a 5-⭐️ 2bdrm   118971   
3  135465             Garden apartment close to Manhattan   663879   
4  169002                Modern Space in Charming Pre-war   805344   

     host_name neighbourhood_group       neighbourhood  latitude  longitude  \
0   Christiana            Brooklyn        Williamsburg  40.70984  -73.95775   
1       Shelly            Brooklyn  Bedford-Stuyvesant  40.68373  -73.92377   
2       Evelyn            Brooklyn         South Slope  40.66552  -73.99019   
3  Christopher            Brooklyn         Fort Greene  40.68626  -73.97598   
4         Alec           Manhattan              Harlem  40.82411  -73.94934   

         room_type  price  minimum_nights  number_of_reviews last_review  \
0  Entire home/apt    100               5                168  2018-07-22   
1  Entire home/apt    125              90                162  2019-06-28   
2  Entire home/apt    169               2                398  2019-06-28   
3  Entire home/apt    170               3                 86  2019-06-20   
4     Private room     65               2                 41  2019-06-16   

   reviews_per_month  calculated_host_listings_count  availability_365  
0               1.57                               1                 0  
1               1.46                               2               137  
2               3.97                               3               182  
3               0.91                               2               286  
4               0.43                               2                59

# Q1. Proprietários com mais imóveis na plataforma

Obtenha os 3 proprietários ("hosts") com mais imóveis sendo anunciados na plataforma, e a respectiva quantidade de imóveis anunciados.


In [6]:
df.columns

Index(['id', 'name', 'host_id', 'host_name', 'neighbourhood_group',
       'neighbourhood', 'latitude', 'longitude', 'room_type', 'price',
       'minimum_nights', 'number_of_reviews', 'last_review',
       'reviews_per_month', 'calculated_host_listings_count',
       'availability_365'],
      dtype='object')

In [13]:
qtd_imoveis_proprietario = df['host_name'].value_counts().sort_values(ascending=False)
qtd_imoveis_proprietario.head(3)

host_name
Sonder (NYC)    7
Mark            7
Andrew          6
Name: count, dtype: int64

# Q2A. Locatários profissionais $\times$ anunciantes com um único imóvel

Considere o preço médio dos imóveis. Qual a diferença do preço médio dos imóveis dos anunciantes com mais de um imóvel e dos anunciantes com um único imóvel anunciado na plataforma?

In [15]:
host_count = df.groupby('host_id')['id'].count()

hosts_um = host_count[host_count == 1].index
hosts_pro = host_count[host_count > 1].index

preco_medio_um = df[df['host_id'].isin(hosts_um)]['price'].mean()
preco_medio_pro = df[df['host_id'].isin(hosts_pro)]['price'].mean()

print(f"Preço médio dos anunciantes com um imóvel: {preco_medio_um}")
print(f"Preço médio dos anunciantes com mais de um imóvel: {preco_medio_pro}")

print( f'Diferença de preços: USD {preco_medio_pro - preco_medio_um:.2f}')

Preço médio dos anunciantes com um imóvel: 150.34954007884363
Preço médio dos anunciantes com mais de um imóvel: 218.1627906976744
Diferença de preços: USD 67.81


# Q3A.  Manhattan *outliers*

Qual o percentual de imóveis de Manhattan com preços estatisticamente discrepantes (*outliers*) com relação aos preços de todos os imóveis?



In [20]:
Q1 = df['price'].quantile(0.25)
Q3 = df['price'].quantile(0.75)
IQR = Q3 - Q1

limite_inf = Q1 - 1.5 * IQR
limite_sup = Q3 + 1.5 * IQR

outliers_all = df[(df['price'] < limite_inf) | (df['price'] > limite_sup)]

manhattan_df = df[df['neighbourhood_group'] == 'Manhattan']

outliers_manhattan = manhattan_df[(manhattan_df['price'] < limite_inf) | (manhattan_df['price'] > limite_sup)]

perc_outliers_manhattan = (len(outliers_manhattan) / len(manhattan_df)) * 100

print( f'Percentuais de outliers de preços, Manhattan: {perc_outliers_manhattan:.2f} %')


Percentuais de outliers de preços, Manhattan: 10.05 %


# Q4A. Preços $\times$ Descrições

Considere as descrições dos anúncios em `df.name`. Crie uma coluna `df['word']` que contém 'beautiful','sunny','spacious' ou 'heart' de acordo com a presença dessa palavra no anúncio. Caso não tenha nenhuma das palavras, assinale 'others'. Ao final, verifique a quantidade e o preço médio dos imóveis para cada grupo dessas palavras nos anúncios.

Atenção, considere a ordem 'beautiful','sunny','spacious' ou 'heart' (isto é, se o texto contém duas ou mais palavras, por exemplo 'beautiful' e 'sunny', considere 'beautiful'). Considere também quaisquer combinações de maiúsculas/minúsculas nas palavras.

In [32]:
df['word'] = 'others'
for word in ['beautiful', 'sunny', 'spacious', 'heart']:
    df.loc[df['name'].str.contains(word, case=False, na=False) & (df['word'] == 'others'), 'word'] = word

df.groupby('word').agg({'price': lambda x: round(x.mean(), 2), 'id': 'count'})


price   id
word                  
beautiful  137.93   41
heart      166.39   36
others     157.89  624
spacious   128.52   60
sunny      137.67   43

# Q5. Enriquecendo os dados

Enriqueça os dados dos imóveis com os valores dos impostos e multas municipais. Cacule a média de impostos e multas de cada grupo de vizinhaça, `neighbourhood_group`. Note que existem imóveis sem impostos e sem multas, esses são imóveis isentos, mas devem ser considerados nos cálculos de média (por exemplo, você pode considerar esses imóveis com valor 0).

In [33]:
df_taxes = pd.read_csv(path + 'AB_NYC_2019_taxes.csv')
df_taxes

id   type   value
0     35907127  taxes  1315.0
1       804815  taxes     NaN
2     22328585  taxes   900.0
3     27876600  taxes   869.0
4     34976037  taxes  1629.0
...        ...    ...     ...
1523  34284316  fines     NaN
1524  20325804  fines     NaN
1525  25872986  fines     NaN
1526  19365218  fines   906.0
1527   4235843  fines     NaN

[1528 rows x 3 columns]

In [51]:
df_airbnb_taxes = df.merge(df_taxes, on='id', how='inner')
df_airbnb_taxes.loc[df_airbnb_taxes['value'].isna(), 'value'] = 0
df_airbnb_taxes.head()

id                                            name  host_id   host_name  \
0  20913              Charming 1 bed GR8 WBurg LOCATION!    79402  Christiana   
1  20913              Charming 1 bed GR8 WBurg LOCATION!    79402  Christiana   
2  25235               Large 2 Bedroom Great for Groups!    87773      Shelly   
3  25235               Large 2 Bedroom Great for Groups!    87773      Shelly   
4  76761  Eveland the Place to Stay & Enjoy a 5-⭐️ 2bdrm   118971      Evelyn   

  neighbourhood_group       neighbourhood  latitude  longitude  \
0            Brooklyn        Williamsburg  40.70984  -73.95775   
1            Brooklyn        Williamsburg  40.70984  -73.95775   
2            Brooklyn  Bedford-Stuyvesant  40.68373  -73.92377   
3            Brooklyn  Bedford-Stuyvesant  40.68373  -73.92377   
4            Brooklyn         South Slope  40.66552  -73.99019   

         room_type  price  minimum_nights  number_of_reviews last_review  \
0  Entire home/apt    100               5                168  2018-07-22   
1  Entire home/apt    100               5                168  2018-07-22   
2  Entire home/apt    125              90                162  2019-06-28   
3  Entire home/apt    125              90                162  2019-06-28   
4  Entire home/apt    169               2                398  2019-06-28   

   reviews_per_month  calculated_host_listings_count  availability_365  \
0               1.57                               1                 0   
1               1.57                               1                 0   
2               1.46                               2               137   
3               1.46                               2               137   
4               3.97                               3               182   

     word   type   value  
0  others  taxes   789.0  
1  others  fines   796.0  
2  others  taxes  1779.0  
3  others  fines     0.0  
4  others  taxes  1516.0

In [52]:
df_airbnb_taxes.groupby(['neighbourhood_group', 'type']).agg({'value': lambda x: int(x.mean())})

value
neighbourhood_group type        
Bronx               fines    126
                    taxes    851
Brooklyn            fines    314
                    taxes    858
Manhattan           fines    314
                    taxes    888
Queens              fines    322
                    taxes    804
Staten Island       fines    581
                    taxes   1157

# Q6. Quebras

Construa no `Pandas` um "relatório" (`df_report`) como abaixo. Os valores são ordenados `neighbourhood_group`,`neighbourhood`. Cada "página" contém apenas 30 linhas. A cada "quebra" de `neighbourhood_group` é gerada uma linha com preço médio do `neighbourhood_group` e, então, é iniciada uma nova página.

O item `line_id` é opcional mas deve auxiliá-lo nessa tarefa.

Ao final produza a soma dos preços das páginas abaixo...

In [66]:
df_reduzido = df[['neighbourhood_group', 'neighbourhood', 'price']].copy()
df_reduzido.head()

neighbourhood_group       neighbourhood  price
0            Brooklyn        Williamsburg    100
1            Brooklyn  Bedford-Stuyvesant    125
2            Brooklyn         South Slope    169
3            Brooklyn         Fort Greene    170
4           Manhattan              Harlem     65

In [74]:
df_mean = df_reduzido.groupby('neighbourhood_group')['price'].mean().reset_index()

df_mean['neighbourhood'] = 'MEAN'

df_mean.head()

df_report = pd.concat([df_reduzido, df_mean]).sort_values(by=['neighbourhood_group', 'neighbourhood']).reset_index(drop=True)

df_report.head(30)

neighbourhood_group       neighbourhood   price
0                Bronx             Belmont    29.0
1                Bronx   Claremont Village    67.0
2                Bronx           Concourse    40.0
3                Bronx            Edenwald    90.0
4                Bronx            Edenwald    80.0
5                Bronx           Fieldston    85.0
6                Bronx           Fieldston   109.0
7                Bronx             Fordham    79.0
8                Bronx          Highbridge    22.0
9                Bronx                MEAN    71.5
10               Bronx             Melrose    69.0
11               Bronx         Morris Park    40.0
12               Bronx          Mott Haven   100.0
13               Bronx         Port Morris    28.0
14               Bronx           Riverdale    99.0
15               Bronx            Van Nest   107.0
16               Bronx           Wakefield   120.0
17               Bronx           Wakefield   120.0
18               Bronx      Williamsbridge    45.0
19               Bronx      Williamsbridge    65.0
20               Bronx      Williamsbridge    36.0
21            Brooklyn           Bay Ridge   100.0
22            Brooklyn           Bay Ridge  1800.0
23            Brooklyn  Bedford-Stuyvesant   125.0
24            Brooklyn  Bedford-Stuyvesant   250.0
25            Brooklyn  Bedford-Stuyvesant    59.0
26            Brooklyn  Bedford-Stuyvesant    45.0
27            Brooklyn  Bedford-Stuyvesant    55.0
28            Brooklyn  Bedford-Stuyvesant    57.0
29            Brooklyn  Bedford-Stuyvesant    59.0

In [79]:
df_report['page_id'] = 0
df_report['line_id'] = 0

page_counter = 1
line_counter = 1

for idx, row in df_report.iterrows():
    if row['neighbourhood'] == 'MEAN':
        group_rows.append((idx, row))
    else:
        group_rows.append((idx, row))

    if len(group_rows) >= 30:
        for i, (row_idx, row) in enumerate(group_rows):
            df_report.at[row_idx, 'page_id'] = page_counter
            df_report.at[row_idx, 'line_id'] = line_counter + i

        page_counter += 1
        line_counter = 1
        group_rows = []

if group_rows:
    for i, (row_idx, row) in enumerate(group_rows):
        df_report.at[row_idx, 'page_id'] = page_counter
        df_report.at[row_idx, 'line_id'] = line_counter + i

df_report.head(30)

neighbourhood_group       neighbourhood   price  page_id  line_id
0                Bronx             Belmont    29.0        1       30
1                Bronx   Claremont Village    67.0        2        1
2                Bronx           Concourse    40.0        2        2
3                Bronx            Edenwald    90.0        2        3
4                Bronx            Edenwald    80.0        2        4
5                Bronx           Fieldston    85.0        2        5
6                Bronx           Fieldston   109.0        2        6
7                Bronx             Fordham    79.0        2        7
8                Bronx          Highbridge    22.0        2        8
9                Bronx                MEAN    71.5        2        9
10               Bronx             Melrose    69.0        2       10
11               Bronx         Morris Park    40.0        2       11
12               Bronx          Mott Haven   100.0        2       12
13               Bronx         Port Morris    28.0        2       13
14               Bronx           Riverdale    99.0        2       14
15               Bronx            Van Nest   107.0        2       15
16               Bronx           Wakefield   120.0        2       16
17               Bronx           Wakefield   120.0        2       17
18               Bronx      Williamsbridge    45.0        2       18
19               Bronx      Williamsbridge    65.0        2       19
20               Bronx      Williamsbridge    36.0        2       20
21            Brooklyn           Bay Ridge   100.0        2       21
22            Brooklyn           Bay Ridge  1800.0        2       22
23            Brooklyn  Bedford-Stuyvesant   125.0        2       23
24            Brooklyn  Bedford-Stuyvesant   250.0        2       24
25            Brooklyn  Bedford-Stuyvesant    59.0        2       25
26            Brooklyn  Bedford-Stuyvesant    45.0        2       26
27            Brooklyn  Bedford-Stuyvesant    55.0        2       27
28            Brooklyn  Bedford-Stuyvesant    57.0        2       28
29            Brooklyn  Bedford-Stuyvesant    59.0        2       29

In [80]:
sum_price_pages = df_report[ df_report.page_id.isin([1,24,25,28]) ].groupby('page_id').price.sum().astype(int).sum()

print(f'A soma dos preços das páginas 1, 24, 25 e 28 é: {sum_price_pages}')

A soma dos preços das páginas 1, 24, 25 e 28 é: 14668


#### Aqui um *sample* do report.

In [ ]:
df_report.iloc[-10::,:]

neighbourhood_group  neighbourhood       price  page_id  line_id
795              Queens       Woodside   75.000000       28       17
796              Queens       Woodside  120.000000       28       18
797              Queens           MEAN   92.227848       28       19
798       Staten Island  Randall Manor   99.000000       29        1
799       Staten Island    South Beach   20.000000       29        2
800       Staten Island    South Beach  100.000000       29        3
801       Staten Island     St. George  130.000000       29        4
802       Staten Island      Todt Hill  135.000000       29        5
803       Staten Island  Tompkinsville   59.000000       29        6
804       Staten Island           MEAN   89.428571       29        7